In [2]:
import os
import cv2
import pickle
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

warnings.filterwarnings("ignore")

# Data Path
data_path = r"C:\Users\rmssh\Downloads\ColorClassification"
categories = ['orange', 'Violet', 'red', 'Blue', 'Green', 'Black', 'Brown', 'White']
image_size = 64  # Reduced from 1280 to 64 for better processing

# Prepare Training Data
training_data = []
def create_training_data():
    for category in categories:
        path = os.path.join(data_path, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.resize(img_array, (image_size, image_size))
                img_array = img_array / 255.0  # Normalize
                training_data.append([img_array, class_num])
            except Exception as e:
                continue

create_training_data()

# Convert list to NumPy array
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])

# Reshape for SVM
X = X.reshape(len(X), -1)  # Flatten images

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute Class Weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(categories))}

# Train SVM Model
svc = SVC(kernel='linear', class_weight=class_weights_dict)
svc.fit(X_train, y_train)

# Predictions
y_pred = svc.predict(X_test)

# Accuracy
print("Accuracy on unknown data:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save Model
with open("color_classification.pkl", "wb") as f:
    pickle.dump(svc, f)

print("Model saved as color_classification.pkl")

# Load & Use Model
with open("color_classification.pkl", "rb") as f:
    loaded_model = pickle.load(f)

print("Loaded model accuracy:", accuracy_score(y_test, loaded_model.predict(X_test)))


Accuracy on unknown data: 0.9565217391304348
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         4
           3       0.50      1.00      0.67         1
           5       1.00      0.75      0.86         4
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         3

    accuracy                           0.96        23
   macro avg       0.93      0.96      0.93        23
weighted avg       0.98      0.96      0.96        23

Model saved as color_classification.pkl
Loaded model accuracy: 0.9565217391304348
